In [ ]:
import chess
import chess.svg
from IPython.display import display, SVG
import time

# Tabela wartości figur
piece_values = {
    chess.PAWN: 100,
    chess.KNIGHT: 300,
    chess.BISHOP: 300,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 0
}

# Tabela pozycji figur
# Wartości są przypisane do każdego pola szachownicy dla każdej figury.
# Wartości dodatnie oznaczają korzystną pozycję, a ujemne niekorzystną.
# Pozycje są symetryczne dla białych i czarnych, więc wartości są takie same dla obu kolorów,
# ale z uwzględnieniem, że białe mają swoje wartości na dole
piece_position = {
    chess.PAWN: [
        0,  0,  0,  0,  0,  0,  0,  0,
        50, 50, 50, 50, 50, 50, 50, 50,
        10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 10, 10, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ],

    chess.KNIGHT: [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50,
    ],

    chess.BISHOP: [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20,
    ],

    chess.ROOK: [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        -5,  0,  0,  0,  0,  0,  0, -5,
        0, -50,  0,  5,  5,  0, -50, 0
    ],

    chess.QUEEN: [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
        -5,  0,  5,  5,  5,  5,  0, -5,
        0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ],
    
    chess.KING: [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
        20, 20,  0,  0,  0,  0, 20, 20,
        20, 30, 10,  0,  0, 10, 30, 20
    ]
}


In [ ]:
def evaluate_board(position):
    """
    Ocenia pozycje szachową.
    """ 

    # Sprawdzenie, czy pozycja jest szach-matem
    if position.is_checkmate():
        return -9999 if position.turn else 9999
    
    # Sprawdzenie remisu
    if position.is_stalemate() or position.is_insufficient_material():
        return 0
    
    score = 0
    
    # Ocena materiału i pozycji figur
    for square, piece in position.piece_map().items():
        piece_type = piece.piece_type
        piece_value = piece_values[piece_type]
        
        if piece.color == chess.WHITE:
            mirrored = chess.square_mirror(square)
            score += piece_value
            score += piece_position[piece_type][mirrored] 
        else:
            score -= piece_value
            score -= piece_position[piece_type][square]
    
    # Mobilność - liczba dostępnych ruchów
    original_turn = position.turn
    
    position.turn = chess.WHITE
    white_mobility = len(list(position.legal_moves))
    
    position.turn = chess.BLACK
    black_mobility = len(list(position.legal_moves))
    
    position.turn = original_turn
    
    # Mobilność - bezpośrednio dodajemy różnicę * 10
    score += 10 * (white_mobility - black_mobility)
    
    # Kara za powtórzenia pozycji
    if position.is_repetition(2):
        current_eval = score / 100
        # Jeśli mamy przewagę, powtórzenie to zła rzecz
        if current_eval > 1.0:  # Białe mają przewagę
            score -= 200
        elif current_eval < -1.0:  # Czarne mają przewagę  
            score += 200
    
    # Bezpieczeństwo króla
    if position.is_check():
        if position.turn == chess.WHITE:
            score -= 50  # Białe w szachu
        else:
            score += 50  # Czarne w szachu
    
    return score / 100

In [ ]:
def is_winning_position(board, white):
    """
    Sprawdza, czy dana pozycja jest wygrywająca dla danego gracza.
    """
    
    # Sprawdź, czy gracz ma przewagę materialną
    score = evaluate_board(board)
    
    if white:
        return score > 1.0  # Białe mają przewagę
    else:
        return score < -1.0  # Czarne mają przewagę

def mvv_lva_score(attacker_type, victim_type):
    """Zwraca score bazowany na MVV-LVA: bijemy droższą figurę tańszą."""
    victim_value = piece_values.get(victim_type, 0)
    attacker_value = piece_values.get(attacker_type, 0)
    return victim_value * 10 - attacker_value  # wyższa wartość = lepszy ruch

def move_order(board):
    """
    Sortuje ruchy na podstawie różnych kryteriów:
    - Promocja do hetmana (najwyższy priorytet)
    - Unikanie powtórzeń pozycji (kara za powtórzenia)
    - Szachy (wysoki priorytet)
    - Bicia z oceną MVV-LVA
    - Rozwój figur w debiucie (pierwsze 10 ruchów)
    """
    scored_moves = []

    for move in board.legal_moves:
        score = 0

        # Promocja do hetmana - najwyższy priorytet
        if move.promotion == chess.QUEEN:
            score += 10000

        # Mocno karze powtórzenia jeśli mamy przewagę
        board.push(move)
        if board.is_repetition(2):
            current_score = evaluate_board(board)
            # Jeśli mamy przewagę, to unikamy powtórzeń
            if (board.turn and current_score > 0.5) or (not board.turn and current_score < -0.5):
                score -= 8000  # Bardzo duża kara
            else:
                score -= 2000  # Mniejsza kara jeśli jesteśmy w gorszej pozycji
        board.pop()

        # Szachy - wysoki priorytet
        board.push(move)
        if board.is_check():
            score += 500
        board.pop()

        # Bicia z oceną MVV-LVA
        if board.is_capture(move):
            attacker = board.piece_at(move.from_square)
            victim = board.piece_at(move.to_square)
            if attacker and victim:
                score += mvv_lva_score(attacker.piece_type, victim.piece_type)

        # Rozwój figur w debiucie (pierwsze 10 ruchów)
        if len(board.move_stack) < 20:  # Pierwsze 10 ruchów każdej strony
            piece = board.piece_at(move.from_square)
            if piece and piece.piece_type in [chess.KNIGHT, chess.BISHOP]:
                # Bonus za rozwój lekkich figur
                if piece.piece_type == chess.KNIGHT:
                    score += 45
                elif piece.piece_type == chess.BISHOP:
                    score += 35

        scored_moves.append((score, move))

    # Sortuj od najlepszych do najgorszych
    scored_moves.sort(key=lambda x: x[0], reverse=True)
    return [move for _, move in scored_moves]


In [ ]:
# True - ruch białych, False - ruch czarnych
def minimax(position, depth, alpha, beta, maximizingPlayer):
    """    
    Algorytm Minimax z przycinaniem Alfa-Beta.
    """
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None

    best_move = None

    # Korzysta z lepszego porządku ruchów
    ordered_moves = move_order(position)

    if maximizingPlayer:
        maxEval = float('-inf')
        for move in ordered_moves:
            position.push(move)
            evaluation, _ = minimax(position, depth - 1, alpha, beta, False)
            position.pop()

            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move

            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return maxEval, best_move

    else:
        minEval = float('inf')
        for move in ordered_moves:
            position.push(move)
            evaluation, _ = minimax(position, depth - 1, alpha, beta, True)
            position.pop()

            if evaluation < minEval:
                minEval = evaluation
                best_move = move

            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return minEval, best_move

In [ ]:
def game(depth=4, max_moves=150):
    """
    Funkcja do rozgrywania gry.
    """

    board = chess.Board()
    display(SVG(chess.svg.board(board, size=400)))
    white = True  # True - ruch białych, False - ruch czarnych
    move_count = 0

    while not board.is_game_over() and move_count < max_moves:
        print(f"Ruch {move_count + 1}, {'Białe' if white else 'Czarne'}")
        
        # Sprawdź powtórzenia
        if board.is_repetition(2):
            print("Powtórzenie pozycji (2x)! Kolejne powtórzenie = remis")
        if board.is_repetition(3):
            print("Powtórzenie pozycji (3x)! Gra kończy się remisem")
            break
        
        score, move = minimax(board, depth, float('-inf'), float('inf'), white)
        
        if move is None:
            print("Brak dostępnych ruchów!")
            break
            
        print(f"Najlepszy ruch: {move}, Ocena: {score:.2f}")
        board.push(move)
        # Wyświetlaj planszę po każdym ruchu
        display(SVG(chess.svg.board(board, size=400,arrows=[chess.svg.Arrow(move.from_square, move.to_square, color="#fc681fcc")],)))

        # Zmiana gracza dla wyszukiwania najlepszego ruchu
        white = not white
        move_count += 1

    print(f"\nGra zakończona po {move_count} ruchach")
    print(f"Wynik: {board.result()}")

In [ ]:
game()